<a href="https://colab.research.google.com/github/jennahgosciak/nyc_fire_risk/blob/main/00_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
#import seaborn.palettes
#import seaborn.utils
import sys
#from census import Census
#from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart


#root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"

#Change root to the folder where you have the data stored, AR has the group project
root = r"G:/.shortcut-targets-by-id/1hgVHDpl_iZ7es6sKW0QRlC9X6SF58wk7/Machine Learning 4 Cities Final Project"

#Change this to match your folder structure
inp= os.join(root, "Data", "2_intermediate_data")
out= os.join(root, "Data", "3_clean_data")

In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: acs_2019_fuel_mobility.csv, ID: 1w3TaiypzzVSIYD9VvhCqwXaNfzkpj9V1
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: hpd_agg_final.csv, ID: 1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: hpd_per_bbl.csv, ID: 1xXilfI1zEmyEYE_TtMdawcnmnZlTv__Z
Title: hpd_2012_2022_v3.csv, ID: 1HXst8E2ZuPVKvKe_9jmSQwKYppOJjraD
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp
Title: all_vacate.csv, ID: 1WfnT

## Load outcomes (fire vacate orders)

In [4]:
csv = drive.CreateFile({'id': '1caDnIMowquAoJuZi1tDlyEqAS4HhXiU0'})
csv.GetContentFile('fire_vacate_bbl.csv')  

df= pd.read_csv('fire_vacate_bbl.csv')[["bbl", "num_vac_orders"]]
df["bbl"]= df["bbl"].astype(str).str.replace(".0", "", regex = False)

### Load PLUTO

In [5]:
csv = drive.CreateFile({'id': '1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o'})
csv.GetContentFile('pluto_df.csv')  

pluto= pd.read_csv('pluto_df.csv').drop("Unnamed: 0", axis =1)
pluto["bbl"]= pluto["bbl"].astype(str).str.replace(".0", "", regex = False)

C:\Users\rivera30\AppData\Local\Temp\ipykernel_6568\1051601485.py:4: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  pluto= pd.read_csv('pluto_df.csv').drop("Unnamed: 0", axis =1)


In [6]:
pluto["boroct2010"]= pluto["boroct2010"].astype(str)
pluto.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,bldgclass_RX,bldgclass_RZ,bldgclass_S0,bldgclass_S1,bldgclass_S2,bldgclass_S3,bldgclass_S4,bldgclass_S5,bldgclass_S9,bldgclass_Z0
0,MN,1012090027,107,181.0,10025,7 WEST 95 STREET,C2,2,NaN,ROBERT & ANNE RABBINO,...,0,0,0,0,0,0,0,0,0,0
1,MN,1003730016,103,2601.0,10009,326 EAST 4 STREET,C1,2,NaN,FAWN EAST FOURTH STREET LLC,...,0,0,0,0,0,0,0,0,0,0
2,MN,1012727501,105,11201.0,10019,33 WEST 56 STREET,RM,4,NaN,UNAVAILABLE OWNER,...,0,0,0,0,0,0,0,0,0,0
3,MN,1003730043,103,2601.0,10009,321 EAST 3 STREET,D3,3,NaN,PENTHOUSE MANAGEMENT LLC,...,0,0,0,0,0,0,0,0,0,0
4,MN,1012090028,107,181.0,10025,5 WEST 95 STREET,C5,2,NaN,"5 WEST 95TH STREET,",...,0,0,0,0,0,0,0,0,0,0


In [7]:
pluto.loc[pluto["boroct2010"].str.contains("None"), :]
pluto.loc[pluto["bbl"]=="2054170262", 'boroct2010']= '2027402'
pluto.loc[pluto["bbl"]=="4153210230", 'boroct2010']= '4107201'
pluto.loc[pluto["bbl"]=="5053090037", 'boroct2010']= '5015603'
pluto.loc[pluto["bbl"]=="5080050028", 'boroct2010']= '5017016'
pluto.loc[pluto["bbl"]=="4142280210", 'boroct2010']= '4009302'
assert pluto.loc[pluto["boroct2010"].str.contains("None"), :].shape[0] == 0

In [8]:
# left join from pluto to capture all properties
# use df for space constraints
df_pl= pluto.merge(df, how = "left", on = "bbl", indicator = True)
print(df_pl["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_pl["vacate_ind"]= np.where(
            df_pl["_merge"]=="both", 1, 0)
df_pl.loc[df_pl["_merge"]=="left_only", "num_vacate_orders"]= 0

print("\nTab of outcome var (vacate ind)")
print(df_pl[["vacate_ind", "_merge"]].value_counts())
print(df_pl[["num_vacate_orders", "_merge"]].value_counts())
df_pl.drop("_merge", axis = 1, inplace = True)

left_only     762424
both            2604
right_only         0
Name: _merge, dtype: int64

Tab of outcome var (vacate ind)
vacate_ind  _merge   
0           left_only    762424
1           both           2604
dtype: int64
num_vacate_orders  _merge   
0.0                left_only    762424
dtype: int64


### HPD Speculations

In [9]:
csv = drive.CreateFile({'id': '1yf86QR6yv2Z9Q6WNINW87RASUsdHPDl6'})
csv.GetContentFile('hpd_spec.csv')  

hpd_spec= pd.read_csv('hpd_spec.csv').drop("Unnamed: 0", axis = 1)
hpd_spec["bbl"]= hpd_spec["bbl"].astype(str).str.replace(".0", "", regex = False)

In [10]:
hpd_spec.head()

,bbl,speculation
0,4005150031,1
1,4155440010,1
2,4030790086,1
3,1002050023,1
4,4021390001,1


In [11]:
# left join from pluto to HPD
# use df for space constraints
df_hpd_spec= df_pl.merge(hpd_spec, how = "left", on = "bbl", indicator = True)
print(df_hpd_spec["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_hpd_spec["speculation"]= np.where(
            df_hpd_spec["_merge"]=="both", 1, 0)

df_hpd_spec.drop("_merge", axis = 1, inplace = True)

left_only     764679
both             351
right_only         0
Name: _merge, dtype: int64


In [12]:
hpd_spec.head()

,bbl,speculation
0,4005150031,1
1,4155440010,1
2,4030790086,1
3,1002050023,1
4,4021390001,1


### HPD Violations

In [13]:
csv = drive.CreateFile({'id': '1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH'})
csv.GetContentFile('hpd_viol.csv')  

hpd_viol= pd.read_csv('hpd_viol.csv').rename({"BBL":"bbl"}, axis = 1)
hpd_viol

,bbl,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio
0,3042719001,6411,5,4898
1,2024310026,4818,36,3712
2,5012720011,4170,0,2775
3,3013020001,4075,10,3257
4,2037430063,3806,55,3223
...,...,...,...,...
181588,1019430052,1,0,0
181589,1012060017,1,0,0
181590,1018850049,1,0,0
181591,1012520050,1,0,0


In [14]:
#hpd_viol= hpd_viol.drop("Unnamed: 0", axis = 1)
hpd_viol["bbl"]= hpd_viol["bbl"].astype(str)
hpd_viol= hpd_viol.drop(hpd_viol[hpd_viol["bbl"]=="0"].index, axis = 0)

hpd_viol.head()

,bbl,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio
0,3042719001,6411,5,4898
1,2024310026,4818,36,3712
2,5012720011,4170,0,2775
3,3013020001,4075,10,3257
4,2037430063,3806,55,3223


In [15]:
# left join from pluto to HPD
# use df for space constraints
df_hpd_viol= df_pl.merge(hpd_viol, how = "left", on = "bbl", indicator = True)
print(df_hpd_viol["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_hpd_viol.loc[df_hpd_viol["_merge"]=="left_only", "viol_count"]= 0

df_hpd_viol.drop("_merge", axis = 1, inplace = True)

left_only     586852
both          178176
right_only         0
Name: _merge, dtype: int64


In [16]:
df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]]= df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]].fillna(0)

In [17]:
df_hpd_viol[["viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit"]]= \
df_hpd_viol[["tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio"]].div(df_hpd_viol["unitsres"], axis = 0).replace([np.inf, -np.inf], np.nan).fillna(0)
df_hpd_viol[["viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit", "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio", "unitsres"]]

,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,tot_hpd_vio,hpd_fire_vio,hpd_b_c_vio,unitsres
0,0.000000,0.0,0.000000,0.0,0.0,0.0,6
1,1.000000,0.0,0.600000,10.0,0.0,6.0,10
2,0.000000,0.0,0.000000,0.0,0.0,0.0,47
3,0.133333,0.0,0.000000,4.0,0.0,0.0,30
4,1.100000,0.0,0.800000,11.0,0.0,8.0,10
...,...,...,...,...,...,...,...
765023,5.500000,0.0,4.333333,33.0,0.0,26.0,6
765024,4.666667,0.0,3.333333,14.0,0.0,10.0,3
765025,1.545455,0.0,0.363636,17.0,0.0,4.0,11
765026,0.888889,0.0,0.111111,8.0,0.0,1.0,9


### DOB Building Permits

In [18]:
csv = drive.CreateFile({'id': '1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp'})
csv.GetContentFile('dev_perm.csv')  

dev= pd.read_csv('dev_perm.csv', header=[0,1], index_col=[0])['avg_permits'].reset_index()
dev["bbl"]= dev["bbl"].astype(str).str.replace(".0", "", regex = False)
dev.head()

permit_type,bbl,AL,NB
0,1000000000,0.444444,0.0
1,1000000001,1.000000,0.0
2,1000000002,0.111111,0.0
3,1000000003,0.222222,0.0
4,1000000004,0.333333,0.0


In [19]:
# left join from pluto to dev dob permits
# use df for space constraints
df_dev= df_hpd_viol.merge(dev, how = "left", on = "bbl", indicator = True)
print(df_dev["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_dev.loc[df_dev["_merge"]=="left_only", "AL"]= 0
df_dev.loc[df_dev["_merge"]=="left_only", "NB"]= 0

df_dev.drop("_merge", axis = 1, inplace = True)

left_only     742923
both           22105
right_only         0
Name: _merge, dtype: int64


### LL 84

In [20]:
csv = drive.CreateFile({'id': '1NOKtbGyL3_TBSMp_eqybVaQc67t33yuo'})
csv.GetContentFile('ll84_avg.csv')  

ll84= pd.read_csv('ll84_avg.csv').drop({"Unnamed: 0"}, axis = 1)
ll84["bbl"]= ll84["bbl"].astype(str).str.replace(".0", "", regex = False)
ll84= ll84.drop(ll84[ll84["bbl"].apply(len)<10].index, axis = 0)

In [21]:
# left join from pluto to dev dob permits
# use df for space constraints
df_ll84= df_dev.merge(ll84, how = "left", on = "bbl", indicator = True)
print(df_ll84["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
df_ll84.loc[df_ll84["_merge"]=="left_only", "average_energy_usage_mi"]= 1
df_ll84.loc[df_ll84["_merge"]=="both", "average_energy_usage_mi"]= 0
df_ll84.loc[df_ll84["_merge"]=="left_only", "average_energy_usage"]= 0

df_ll84.drop("_merge", axis = 1, inplace = True)

left_only     747641
both           17387
right_only         0
Name: _merge, dtype: int64


### Export analytic file (w/o census tract info)

In [22]:
df_ll84.to_csv(os.join(root, out, "analysis_bbl_noacs.csv"))

In [23]:
df_ll84.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,hpd_fire_vio,hpd_b_c_vio,viol_count,viol_count_per_unit,hpd_fire_vio_per_unit,hpd_b_c_vio_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,MN,1012090027,107,181.0,10025,7 WEST 95 STREET,C2,2,NaN,ROBERT & ANNE RABBINO,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.0
1,MN,1003730016,103,2601.0,10009,326 EAST 4 STREET,C1,2,NaN,FAWN EAST FOURTH STREET LLC,...,0.0,6.0,NaN,1.000000,0.0,0.6,0.0,0.000000,0.000000,1.0
2,MN,1012727501,105,11201.0,10019,33 WEST 56 STREET,RM,4,NaN,UNAVAILABLE OWNER,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,77.366667,0.0
3,MN,1003730043,103,2601.0,10009,321 EAST 3 STREET,D3,3,NaN,PENTHOUSE MANAGEMENT LLC,...,0.0,0.0,NaN,0.133333,0.0,0.0,0.0,0.111111,0.000000,1.0
4,MN,1012090028,107,181.0,10025,5 WEST 95 STREET,C5,2,NaN,"5 WEST 95TH STREET,",...,0.0,8.0,NaN,1.100000,0.0,0.8,0.0,0.000000,0.000000,1.0


## ACS Data, for 2015-2019 5-year

In [30]:
# load acs data (to check census tract recoding)
csv = drive.CreateFile({'id': '17PvjIPesTzugausfzMi7rTiXahTiHaH1'})
csv.GetContentFile('acs_data.csv')  

acs= pd.read_csv('acs_data.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
acs= acs.loc[acs['year']==2019]
acs.head()

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhblack,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter
21721,36005000100,6864,10599.550,0.647574,6345,519,103,3322,3392,47,...,52.462121,0.262238,2.636946,0.0,0.276807,0.509907,NaN,NaN,NaN,NaN
21722,36005000200,4532,24481.190,0.185122,2193,2339,1134,862,1605,931,...,20.300088,0.000000,1.654898,0.0,1.323919,0.000000,52.459016,47.540984,11.997019,18.181818
21723,36005000400,5516,18537.140,0.297565,2810,2706,1071,1319,2362,764,...,22.915156,0.000000,2.193619,0.0,0.163162,1.341552,79.040157,20.959843,1.860921,4.906542
21724,36005001600,5825,31101.490,0.187290,2422,3403,1546,1399,1978,902,...,34.695279,0.000000,1.098712,0.0,0.858369,0.429185,12.430238,87.569762,11.415525,12.514484
21725,36005001900,3141,4877.152,0.644023,1583,1558,898,1052,1111,80,...,29.831264,0.000000,2.292264,0.0,0.700414,1.782872,10.565338,89.434662,9.638554,10.777202


### ACS Data, fuel and mobility variables

In [36]:
csvnew = drive.CreateFile({'id': '1w3TaiypzzVSIYD9VvhCqwXaNfzkpj9V1'})
csvnew.GetContentFile('acs_2019_fuel_mobility.csv')  

acs_new= pd.read_csv('acs_2019_fuel_mobility.csv').drop(['Unnamed: 0'], axis=1)

                                
acs_new

,Geo_FIPS,pcthgas,pcthelectricity,pcthoilker,pcthcoal,pcthsolar,pcthotherf,pcthnofuel,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pop1yrplusinhh,pctpoprenterhh
0,36005000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,36005000200,75.037258,11.549925,13.412817,0.000000,0.000000,0.00000,0.000000,97.917631,0.601573,0.555298,0.925497,0.000000,4515,0.297231
2,36005000400,68.511263,16.748286,14.054848,0.000000,0.000000,0.00000,0.685602,100.000000,0.000000,0.000000,0.000000,0.000000,5450,0.374679
3,36005001600,40.334855,12.988331,42.465753,0.000000,0.000000,0.00000,4.211060,96.144227,2.021676,1.042101,0.270946,0.521050,5561,0.354433
4,36005001900,43.836886,23.076923,25.949954,0.000000,0.000000,4.07785,3.058387,88.834951,3.353928,5.251545,2.250662,0.308914,2655,0.406403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,36085030302,91.872792,4.896517,2.170621,1.060071,0.000000,0.00000,0.000000,99.170555,0.829445,0.000000,0.000000,0.000000,6594,0.300425
2163,36085031901,92.768792,1.712655,5.518554,0.000000,0.000000,0.00000,0.000000,98.778004,1.221996,0.000000,0.000000,0.000000,2792,0.376433
2164,36085031902,89.762340,5.606338,2.132846,0.000000,1.828154,0.00000,0.670323,97.538337,1.533495,0.282486,0.000000,0.645682,5255,0.312274
2165,36085032300,88.913525,10.199557,0.886918,0.000000,0.000000,0.00000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,1073,0.420317


In [66]:
acs_new = acs_new.rename(columns={'Geo_FIPS': 'geofips'})


In [67]:
df_acs= pd.merge(acs, acs_new, on=["geofips"])
df_acs.head()

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pcthsolar,pcthotherf,pcthnofuel,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pop1yrplusinhh,pctpoprenterhh
0,36005000100,6864,10599.550,0.647574,6345,519,103,3322,3392,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,36005000200,4532,24481.190,0.185122,2193,2339,1134,862,1605,931,...,0.0,0.00000,0.000000,97.917631,0.601573,0.555298,0.925497,0.000000,4515,0.297231
2,36005000400,5516,18537.140,0.297565,2810,2706,1071,1319,2362,764,...,0.0,0.00000,0.685602,100.000000,0.000000,0.000000,0.000000,0.000000,5450,0.374679
3,36005001600,5825,31101.490,0.187290,2422,3403,1546,1399,1978,902,...,0.0,0.00000,4.211060,96.144227,2.021676,1.042101,0.270946,0.521050,5561,0.354433
4,36005001900,3141,4877.152,0.644023,1583,1558,898,1052,1111,80,...,0.0,4.07785,3.058387,88.834951,3.353928,5.251545,2.250662,0.308914,2655,0.406403


In [68]:
df_acs['geofips_recode']= df_acs['geofips'].astype(str).str[2:]
df_acs['geofips_recode']= np.select([df_acs['geofips_recode'].str[0:3] == '005',
                                 df_acs['geofips_recode'].str[0:3] == '047',
                                 df_acs['geofips_recode'].str[0:3] == '061',
                                 df_acs['geofips_recode'].str[0:3] == '081',
                                 df_acs['geofips_recode'].str[0:3] == '085'],
                                ['2' + df_acs['geofips_recode'].str[3:],
                                '3' + df_acs['geofips_recode'].str[3:],
                                 '1' + df_acs['geofips_recode'].str[3:],
                                 '4' + df_acs['geofips_recode'].str[3:],
                                 '5' + df_acs['geofips_recode'].str[3:]])

print(df_acs.loc[df_acs['geofips_recode'].str.len() < 7, 'geofips_recode'].head())
df_acs.loc[df_acs['geofips_recode'].str.len() < 7, 'geofips_recode']= \
df_acs.loc[df_acs['geofips_recode'].str.len() < 7, 'geofips_recode'].str.pad(width = 7, fillchar = "0", side = "right")


df_acs["geofips_recode"]= df_acs["geofips_recode"].str.replace(".0", "", regex = False)

df_acs[['geofips_recode', 'geofips']].head()


Series([], Name: geofips_recode, dtype: object)


,geofips_recode,geofips
0,2000100,36005000100
1,2000200,36005000200
2,2000400,36005000400
3,2001600,36005001600
4,2001900,36005001900


In [54]:
acs.loc[acs["geofips_recode"]=="3114201", ["medianhhincome", "medianhhincome_imp"]]

In [69]:
df_ll84["boroct2010"]= df_ll84["boroct2010"].astype(str)
df_ll84["boroct2010"]

0         1018100
1         1002601
2         1011201
3         1002601
4         1018100
           ...   
765023    1010400
765024    1010400
765025    1002601
765026    1018100
765027    1011201
Name: boroct2010, Length: 765028, dtype: object

In [70]:
df_acs2= pd.merge(df_ll84, df_acs, how = "outer", left_on =  "boroct2010", right_on = "geofips_recode", indicator = True)
df_acs2.head()

,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,pcthotherf,pcthnofuel,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pop1yrplusinhh,pctpoprenterhh,_merge
0,MN,1012090027,107.0,181.0,10025.0,7 WEST 95 STREET,C2,2.0,NaN,ROBERT & ANNE RABBINO,...,6.074109,4.502814,91.043879,3.285915,4.307754,1.362452,0.0,8416.0,0.506654,both
1,MN,1012090028,107.0,181.0,10025.0,5 WEST 95 STREET,C5,2.0,NaN,"5 WEST 95TH STREET,",...,6.074109,4.502814,91.043879,3.285915,4.307754,1.362452,0.0,8416.0,0.506654,both
2,MN,1012090037,107.0,181.0,10025.0,12 WEST 96 STREET,D4,3.0,NaN,12 W 96TH STREET CORP,...,6.074109,4.502814,91.043879,3.285915,4.307754,1.362452,0.0,8416.0,0.506654,both
3,MN,1012090049,107.0,181.0,10025.0,44 WEST 96 STREET,D4,3.0,NaN,44 OWNERS CORP,...,6.074109,4.502814,91.043879,3.285915,4.307754,1.362452,0.0,8416.0,0.506654,both
4,MN,1012090051,107.0,181.0,10025.0,46 WEST 96 STREET,D4,3.0,NaN,46 W 96TH ST OWNERS CORP,...,6.074109,4.502814,91.043879,3.285915,4.307754,1.362452,0.0,8416.0,0.506654,both


In [71]:
print(df_acs2["_merge"].value_counts())
df_acs2= df_acs2[df_acs2["_merge"] == "both"]
df_acs2= df_acs2.drop("_merge", axis = 1)

both          765026
right_only        57
left_only          2
Name: _merge, dtype: int64


In [73]:
## impute missing census tract values with the median for the whole city
mid_citywide= df_acs2.loc[:, "totpop":].median(axis = 0).rename('median').reset_index()
print(mid_citywide)

for i in mid_citywide['index']:
    df_acs2.loc[df_acs2[i].isna(), i]= mid_citywide.loc[mid_citywide['index']==i, 'median'].values[0]

                index        median
0              totpop   3772.000000
1             popdens  31972.500000
2          areasqmile      0.087706
3               males   1845.000000
4             females   1965.000000
..                ...           ...
62  pctmvdsdiffcounty      0.951626
63   pctmvdsdiffstate      0.000000
64       pctmvdabroad      0.000000
65     pop1yrplusinhh   3695.000000
66     pctpoprenterhh      0.350567

[67 rows x 2 columns]


In [74]:
for i in df_acs2.columns:
    if df_acs2[i].isna().sum() > 0:
        print(i)
        print(df_acs2[i].isna().sum())

tract2010
3
address
27
ownertype
757695
ownername
15
sanborn
38
holc_AB
353
holc_CD
353
holc_D
353
num_vac_orders
762422
num_vacate_orders
2604
viol_count
178176
average_energy_usage
435


In [75]:
list(df_acs2.columns)

['borough',
 'bbl',
 'cd',
 'tract2010',
 'zipcode',
 'address',
 'bldgclass',
 'landuse',
 'ownertype',
 'ownername',
 'lotarea',
 'bldgarea',
 'numbldgs',
 'numfloors',
 'unitsres',
 'unitstotal',
 'assessland',
 'assesstot',
 'exempttot',
 'yearbuilt',
 'yearalter1',
 'yearalter2',
 'sanborn',
 'holc_AB',
 'holc_CD',
 'holc_D',
 'boroct2010',
 'landuse_01',
 'landuse_02',
 'landuse_03',
 'landuse_04',
 'bldgclass_A0',
 'bldgclass_A1',
 'bldgclass_A2',
 'bldgclass_A3',
 'bldgclass_A4',
 'bldgclass_A5',
 'bldgclass_A6',
 'bldgclass_A7',
 'bldgclass_A8',
 'bldgclass_A9',
 'bldgclass_B1',
 'bldgclass_B2',
 'bldgclass_B3',
 'bldgclass_B9',
 'bldgclass_C0',
 'bldgclass_C1',
 'bldgclass_C2',
 'bldgclass_C3',
 'bldgclass_C4',
 'bldgclass_C5',
 'bldgclass_C6',
 'bldgclass_C7',
 'bldgclass_C8',
 'bldgclass_C9',
 'bldgclass_CM',
 'bldgclass_D0',
 'bldgclass_D1',
 'bldgclass_D2',
 'bldgclass_D3',
 'bldgclass_D4',
 'bldgclass_D5',
 'bldgclass_D6',
 'bldgclass_D7',
 'bldgclass_D8',
 'bldgclass_D9

In [78]:
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi", "landuse_cat", "bldgclass_cat"]
predictors_acs= df_acs2.columns[df_acs2.columns.get_loc("totpop"):df_acs2.columns.get_loc("pctpoprenterhh") + 1].tolist()
landusecols = ["landuse_" + '0' + str(x) for x in range(1,5)]
bldgclasscols = ["bldgclass_" + str(x) for x in df_acs2["bldgclass"].unique()]

predictors_full= predictors + predictors_acs + landusecols + bldgclasscols

In [80]:
# create categorical variables
df_acs2["landuse_cat"]= pd.Categorical(df_acs2["landuse"])
df_acs2["bldgclass_cat"]= pd.factorize(df_acs2['bldgclass'])[0]
df_acs2.columns

Index(['borough', 'bbl', 'cd', 'tract2010', 'zipcode', 'address', 'bldgclass',
       'landuse', 'ownertype', 'ownername',
       ...
       'pcthnofuel', 'pctsamehouse', 'pctmvdsamecounty', 'pctmvdsdiffcounty',
       'pctmvdsdiffstate', 'pctmvdabroad', 'pop1yrplusinhh', 'pctpoprenterhh',
       'landuse_cat', 'bldgclass_cat'],
      dtype='object', length=173)

In [81]:
df_final= df_acs2.loc[:, predictors_full + ["bbl", "boroct2010", "vacate_ind", "borough", "landuse", "bldgclass"]]
df_final.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,borough,landuse,bldgclass
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.0,1012090027,1018100,0.0,MN,2.0,C2
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,0.0,0.0,0.0,0.0,1012090028,1018100,0.0,MN,2.0,C5
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.0,1012090037,1018100,0.0,MN,3.0,D4
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,0.0,0.0,0.0,0.0,1012090049,1018100,0.0,MN,3.0,D4
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,0.0,0.0,0.0,0.0,1012090051,1018100,0.0,MN,3.0,D4


In [82]:
df_final.loc[df_final["holc_AB"].isna(), "holc_AB"]= 0
df_final.loc[df_final["holc_CD"].isna(), "holc_CD"]= 0
df_final.loc[df_final["holc_D"].isna(), "holc_D"]= 0
df_final.loc[df_final["average_energy_usage"].isna(), "average_energy_usage_mi"]= 1
df_final.loc[df_final["average_energy_usage"].isna(), "average_energy_usage"]= 0

In [83]:
print(df_final.dropna().shape)
print(df_final.shape)

(765026, 160)
(765026, 160)


In [84]:
assert df_final.dropna().shape[0] == df_final.shape[0]

## Export analytic file

In [87]:
df_final.to_csv(os.join(root, out, "analysis_bbl.csv"))